<a href="https://colab.research.google.com/github/TranQuocHuy83/RAG/blob/colab-dev/notebooks/RAG_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Đánh giá RAG**
_Tác giả: [Aymeric Roucher](https://huggingface.co/m-ric)_

Sổ tay này minh họa cách bạn có thể đánh giá hệ thống RAG (Retrieval Augmented Generation-Thế hệ tăng cường truy suất) của mình bằng cách xây dựng một tập dữ liệu đánh giá tổng hợp và sử dụng LLM làm công cụ đánh giá để tính toán độ chính xác của hệ thống.

Để tìm hiểu thêm về RAG, bạn có thể xem [sổ tay khác này](rag_zephyr_langchain)!

Hệ thống RAG rất phức tạp: đây là sơ đồ RAG, trong đó chúng tôi đã đánh dấu màu xanh lam tất cả các khả năng cải tiến hệ thống:

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png" height="700">

Việc triển khai bất kỳ cải tiến nào trong số này đều có thể mang lại hiệu suất tăng đáng kể; nhưng việc thay đổi bất cứ điều gì cũng vô ích nếu bạn không thể theo dõi tác động của những thay đổi đó đến hiệu suất của hệ thống!

Vậy hãy cùng xem cách đánh giá hệ thống RAG của chúng ta.

### **Đánh giá hiệu suất của RAG**

Vì có rất nhiều thành phần cần điều chỉnh và ảnh hưởng lớn đến hiệu suất, việc đánh giá hiệu năng của hệ thống RAG là rất quan trọng.

Để xây dựng quy trình đánh giá, chúng ta cần:
1. Một tập dữ liệu đánh giá với các cặp câu hỏi - câu trả lời (cặp QA)
2. Một bộ đánh giá để tính toán độ chính xác của hệ thống trên tập dữ liệu đánh giá ở trên.

➡️ Hóa ra, chúng ta có thể sử dụng LLM để hỗ trợ trong suốt quá trình!

1. Tập dữ liệu đánh giá sẽ được tạo ra một cách tổng hợp bởi một LLM 🤖, và các câu hỏi sẽ được lọc ra bởi các LLM khác 🤖
2. Một tác nhân [LLM-as-a-judge](https://huggingface.co/papers/2306.05685) 🤖 sau đó sẽ thực hiện đánh giá trên tập dữ liệu tổng hợp này.

__Hãy cùng tìm hiểu và bắt đầu xây dựng quy trình đánh giá của chúng ta!__ Đầu tiên, chúng ta cài đặt các phụ thuộc mô hình cần thiết.

In [1]:
!pip install -q torch transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille
!pip install -U langchain-text-splitters langchain-community langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 126.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9

In [2]:
# %reload_ext autoreload
# %autoreload 2

In [3]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None) #Hiển thị trọn vẹn nội dung văn bản

In [4]:
from huggingface_hub import notebook_login

notebook_login()

### **Load your knowledge base**

In [5]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

In [6]:
print(ds)

Dataset({
    features: ['text', 'source'],
    num_rows: 2647
})


In [7]:
import pandas as pd

# Chuyển 5 dòng đầu tiên sang dạng bảng
df_view = pd.DataFrame(ds.select(range(5)))
display(df_view)

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# **1. Xây dựng bộ dữ liệu tổng hợp để đánh giá**
Trước tiên, chúng ta xây dựng một bộ dữ liệu tổng hợp gồm các câu hỏi và ngữ cảnh liên quan. Phương pháp là lấy các phần tử từ cơ sở tri thức của chúng ta, và yêu cầu một LLM tạo ra các câu hỏi dựa trên các tài liệu này.

Sau đó, chúng ta thiết lập các tác nhân LLM khác để hoạt động như các bộ lọc chất lượng cho các cặp câu hỏi-trả lời được tạo ra: mỗi tác nhân sẽ hoạt động như một bộ lọc cho một lỗi cụ thể.

### **1.1. Prepare source documents**

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document as LangchainDocument

langchain_docs = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = [] # danh sách văn bản ngắn --> chuẩn bị chuyển thành vector
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])
# kiểm tra số lượng văn bản
print(len(docs_processed))

  0%|          | 0/2647 [00:00<?, ?it/s]

28308


### **1.2. Thiết lập tác nhân để tạo câu hỏi**

Chúng tôi sử dụng [Mixtral](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1) để tạo cặp câu hỏi QA vì nó có hiệu suất tuyệt vời trên các bảng xếp hạng như [Chatbot Arena](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard).

In [9]:
from huggingface_hub import InferenceClient

# Khai báo mô hình
# repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# repo_id = "HuggingFaceH4/zephyr-7b-beta"
repo_id = "meta-llama/Llama-3.2-3B-Instruct"

llm_client = InferenceClient(model=repo_id, timeout=120) # công cụ gửi yêu cầu lên server


def call_llm(inference_client: InferenceClient, prompt: str):
  # Nhận:
  # inference_client → kết nối server
  # prompt → nội dung người dùng

    # Sử dụng chat_completion thay vì text_generation
    messages1 = [{"role": "user", "content": prompt}]

    response = inference_client.chat_completion(
        messages=messages1,
        max_tokens=200,
        temperature=0.1
    )
    return response.choices[0].message.content

# Chạy thử nghiệm
try:
    result = call_llm(llm_client, "Việt Nam có bao nhiêu tỉnh/thành?")
    print(result)
except Exception as e:
    print(f"Lỗi: {e}")

Việt Nam hiện có 63 tỉnh, thành phố.


In [10]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [11]:
# 2. Thử nghiệm với một đoạn văn bản thật trong docs_processed
import random

# Lấy ngẫu nhiên 1 đoạn trong số 28.308 đoạn đã chia
random_idx = random.randint(0, len(docs_processed)-1)
sample_context = docs_processed[random_idx].page_content

# Gửi cho AI
print(f"--- ĐOẠN VĂN GỐC ---\n{sample_context}\n")
result = call_llm(llm_client, QA_generation_prompt.format(context=sample_context))
print(f"--- AI TỰ ĐẶT CÂU HỎI ---\n{result}")

--- ĐOẠN VĂN GỐC ---
```py
import soundfile
from espnet2.bin.tts_inference import Text2Speech

text2speech = Text2Speech.from_pretrained("model_name")
speech = text2speech("foobar")["wav"]
soundfile.write("out.wav", speech.numpy(), text2speech.fs, "PCM_16")
```

If you want to see how to load a specific model, you can click `Use in ESPnet` and you will be given a working snippet that you can load it! 

<div class="flex justify-center">
<img class="block dark:hidden" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/hub/libraries-espnet_snippet.png"/>
<img class="hidden dark:block" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/hub/libraries-espnet_snippet-dark.png"/>
</div>

## Sharing your models

`ESPnet` outputs a `zip` file that can be uploaded to Hugging Face easily. For a full guide on sharing  models, we recommend checking out the [official guide](https://github.com/espnet/espnet_model_zoo#register-your-model))

Bây giờ chúng ta hãy tạo các cặp kiểm thử chất lượng (QA).

Trong ví dụ này, chúng ta chỉ tạo 5 cặp QA và sẽ tải phần còn lại từ Hub.

Nhưng đối với cơ sở tri thức cụ thể của bạn, giả sử bạn muốn có ít nhất khoảng 100 mẫu thử nghiệm, và tính đến việc chúng ta sẽ lọc bỏ khoảng một nửa số này bằng các tác nhân phê bình sau này, bạn nên tạo nhiều hơn, khoảng >200 mẫu.

In [12]:
import random
import re

N_GENERATIONS = 5  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    raw_output = call_llm(
        llm_client, QA_generation_prompt.format(context=sampled_context.page_content)
    )

    try:
        # Regex này sẽ tìm nội dung nằm sau 'Question:' và 'Answer:' bất kể viết hoa hay thường
        question_match = re.search(r"(?:Question|Factoid question):\s*(.*)", raw_output, re.IGNORECASE)
        answer_match = re.search(r"Answer:\s*(.*)", raw_output, re.IGNORECASE)

        if question_match and answer_match:
            question = question_match.group(1).split("Answer:")[0].strip()
            answer = answer_match.group(1).strip()

            # Chỉ lưu nếu câu trả lời không quá dài và không bị rỗng
            if 10 < len(answer) < 500:
                outputs.append({
                    "context": sampled_context.page_content,
                    "question": question,
                    "answer": answer,
                    "source_doc": sampled_context.metadata["source"],
                })
    except Exception as e:
        continue

Generating 5 QA couples...


  0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
print(outputs)

[{'context': "First, you have to abandon all hope to have exactly the same logits at the end down\nto the bytes. PyTorch versions can change the kernels and introduce subtle differences, and different hardware\nmight yield different results because of different architecture (and you probably\ndon't want to develop on a A100 GPU all the time for cost reasons).\n\n***Getting a good strict test suite is really important for all models***", 'question': 'What is the main reason for not expecting the same logits at the end of a PyTorch model run on different hardware or versions?', 'answer': 'Different hardware and PyTorch versions can introduce subtle differences due to varying kernels and architectures.', 'source_doc': 'huggingface/blog/blob/main/bloom-inference-optimization.md'}, {'context': '<details open>\n<summary><b>Code:</b></summary>\n\n```python\noutput = model.generate(input_ids, max_length=512, penalty_alpha=0.6, top_k=4)\nprint("Output:\\n" + 100 * \'-\')\nprint(tokenizer.decode

In [14]:
# Chuyển thành DataFrame để dễ quan sát
display(pd.DataFrame(outputs).head(2))

,context,question,answer,source_doc
0,"First, you have to abandon all hope to have exactly the same logits at the end down\nto the bytes. PyTorch versions can change the kernels and introduce subtle differences, and different hardware\nmight yield different results because of different architecture (and you probably\ndon't want to develop on a A100 GPU all the time for cost reasons).\n\n***Getting a good strict test suite is really important for all models***",What is the main reason for not expecting the same logits at the end of a PyTorch model run on different hardware or versions?,Different hardware and PyTorch versions can introduce subtle differences due to varying kernels and architectures.,huggingface/blog/blob/main/bloom-inference-optimization.md
1,"<details open>\n<summary><b>Code:</b></summary>\n\n```python\noutput = model.generate(input_ids, max_length=512, penalty_alpha=0.6, top_k=4)\nprint(""Output:\n"" + 100 * '-')\nprint(tokenizer.decode(output[0], skip_special_tokens=True))\nprint("""" + 100 * '-')\n```\n</details>\n\n<details open>\n<summary><b>Model Output:</b></summary>\n\n```\nOutput:\n----------------------------------------------------------------------------------------------------\nIn a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored\nvalley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns\nspoke perfect English.\n\nAccording to the BBC, a team of scientists led by Dr David MacKay, from the University of Bristol, spent\ntwo years searching for the unicorn herd, which they discovered during a survey of the area.",What is the name of the university led by Dr David MacKay that spent two years searching for the unicorn herd?,The University of Bristol.,huggingface/blog/blob/main/introducing-csearch.md


### **1.3. Thiết lập các tác nhân đánh giá**

Các câu hỏi do tác nhân trước đó tạo ra có thể có nhiều lỗi: chúng ta nên kiểm tra chất lượng trước khi xác nhận các câu hỏi này.

Do đó, chúng ta xây dựng các tác nhân đánh giá sẽ xếp hạng từng câu hỏi dựa trên một số tiêu chí, được nêu trong [bài báo này](https://huggingface.co/papers/2312.10003):
- **Tính xác thực:** câu hỏi có thể được trả lời từ ngữ cảnh đã cho không?

- **Tính phù hợp:** câu hỏi có phù hợp với người dùng không? Ví dụ, `"Ngày phát hành transformers 4.29.1 là khi nào?"` không phù hợp với những người thực hành ML.

Một trường hợp lỗi cuối cùng mà chúng tôi nhận thấy là khi một hàm được thiết kế riêng cho bối cảnh cụ thể nơi câu hỏi được tạo ra, nhưng bản thân nó lại không thể giải mã được, chẳng hạn như `"Tên của hàm được sử dụng trong hướng dẫn này là gì?"`.
Chúng tôi cũng xây dựng một tác nhân phê bình cho các tiêu chí này:
- **Độc lập**: câu hỏi có dễ hiểu khi không có bất kỳ ngữ cảnh nào, đối với người có kiến ​​thức chuyên môn/truy cập Internet không? Ngược lại với điều này là câu hỏi `Hàm nào được sử dụng trong bài viết này?` đối với một câu hỏi được tạo ra từ một bài viết blog cụ thể.

Chúng tôi chấm điểm các hàm một cách có hệ thống bằng tất cả các tác nhân này, và bất cứ khi nào điểm số quá thấp đối với bất kỳ tác nhân nào, chúng tôi sẽ loại bỏ câu hỏi đó khỏi tập dữ liệu đánh giá của mình.

💡 ___Khi yêu cầu các tác nhân đưa ra điểm số, trước tiên chúng tôi yêu cầu chúng đưa ra lý do. Điều này sẽ giúp chúng tôi xác minh điểm số, nhưng quan trọng hơn, việc yêu cầu chúng đưa ra lý do trước tiên sẽ cung cấp cho mô hình nhiều token hơn để suy nghĩ và xây dựng câu trả lời trước khi tóm tắt nó thành một token điểm số duy nhất.___

Bây giờ chúng tôi xây dựng và chạy các tác nhân phê bình này.

In [15]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [16]:
import re

def parse_llm_judge_output(text):
    score_match = re.search(r"Total rating:\s*([1-5])", text, re.IGNORECASE)
    eval_match = re.search(
        r"Evaluation:\s*(.*?)(?:\n\s*Total rating:|$)",
        text,
        re.IGNORECASE | re.DOTALL
    )

    if not score_match or not eval_match:
        raise ValueError("Cannot parse judge output")

    score = int(score_match.group(1))
    evaluation = eval_match.group(1).strip()

    return score, evaluation


In [17]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(
                context=output["context"], question=output["question"]
            ),
        ),
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            llm_client,
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
      for criterion, evaluation in evaluations.items():
        score, eval_text = parse_llm_judge_output(evaluation)

        output.update({
            f"{criterion}_score": score,
            f"{criterion}_eval": eval_text,
        })
    except Exception:
      continue


Generating critique for each QA couple...


  0%|          | 0/2 [00:00<?, ?it/s]

Giờ chúng ta hãy lọc ra những câu hỏi không hay dựa trên điểm số của hệ thống đánh giá:

In [18]:
import pandas as pd

pd.set_option("display.max_colwidth", None)# không ngắt câu hỏi/ câu trả lời

generated_questions = pd.DataFrame.from_dict(outputs) # tạo DataFrame từ outputs

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

eval_dataset = datasets.Dataset.from_pandas(
    generated_questions, split="train", preserve_index=False
)

Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,What is the main reason for not expecting the same logits at the end of a PyTorch model run on different hardware or versions?,Different hardware and PyTorch versions can introduce subtle differences due to varying kernels and architectures.,5,4,5
1,What is the name of the university led by Dr David MacKay that spent two years searching for the unicorn herd?,The University of Bristol.,1,1,1


Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
0,What is the main reason for not expecting the same logits at the end of a PyTorch model run on different hardware or versions?,Different hardware and PyTorch versions can introduce subtle differences due to varying kernels and architectures.,5,4,5
